Read binary output file write out as an ascii file

input file name infile = open('../../smaug_jet_data/out/hydro1/hydro1_36000.out','rb')

output filename outfile = open('../../smaug_jet_data/out/hydro1/hydro1_asc_36000.ini','rb')

In [1]:
import numpy as np
from numpy import *
import scipy.io
from scipy import special

import struct

%matplotlib inline  

In [2]:
infile = open('../../smaug_jet_data/out/hydro4a-10k/hydro4a_1780000.out','rb')
outfile = open('../../smaug_jet_data/out/hydro4a-10k/hydro4a_asc_1780000.ini','wb')

Read input file

In [3]:
file=infile

file.seek(0,2)
eof = file.tell()
file.seek(0,0)

ndata=[0,0]

name = file.read(79)
nit = fromfile(file,dtype=int32,count=1)
t = fromfile(file,dtype=float64,count=1)
ndim=fromfile(file,dtype=int32,count=1)
neqpar=fromfile(file,dtype=int32,count=1)
nw=fromfile(file,dtype=int32,count=1)
ndata[0]=fromfile(file,dtype=int32,count=1)
ndata[1]=fromfile(file,dtype=int32,count=1)
#ndata = fromfile(file,dtype=int32,count=ndim)[:ndim]
varbuf = fromfile(file,dtype=float,count=6)[:6]
varnames = file.read(79)


icount=(nw+ndim)*ndata[0]*ndata[1]
print(icount)
alldat=fromfile(file,dtype=float64,count=25165824)
alldat=np.reshape(alldat,(12,1024,2048),'C')


#typedef enum vars {rho, mom1, mom2, energy, b1, b2,energyb,rhob,b1b,b2b} CEV;

#alldat=fromfile(file,dtype=float,count=(nw+ndim)*ndata[0]*ndata[1])[:(nw+ndim)*ndata[0]*ndata[1]]
#if size(alldat)<(nw+ndim)*ndata[0]*ndata[1]:
#    alldat=resize(alldat,(nw+ndim)*ndata[0]*ndata[1])
#alldat=np.reshape(alldat,(nw+ndim,ndata[0],ndata[1],),'C')
print(shape(alldat))
file.close()

[25165824]
(12, 1024, 2048)


write outputfile

In [4]:
file=outfile

#this script assumes data has been read using a routine such as sac-read3-ascii.py
#the following variables are assumed
#nits
#time
#ndim
#nvar
#nfields

#dim[2] or dim[3]

#gamma
#eta
#grav1
#grav2
#grav3

#all data is contained in an array alldat of shape nfields+ndim,dim[0],dim[1]


#write header lines

#header='sac_test_asc'
##header=modelinfo[0]
#modelinfo=(header,nits, time, ndim, nvar, nfields,dim,head3,head4)
##dim=[128,128]
##ndim=2
##nfields=12
##time=modelinfo[2]
##nits=modelinfo[1]
##nvar=modelinfo[4]

#header='sac_test_asc'
header='hydro1_asc_924000.ini'

dim=[2048,1024]
ndim=2
nfields=12
time=t
nits=nit
nvar=nw


head1=str(nits)+" "+str(time)+" "+str(ndim)+" "+str(nvar)+" "+str(nfields)

if ndim==2:
    head2=str(dim[0])+" "+str(dim[1])
elif ndim==3:
    head2=str(dim[0])+" "+str(dim[1])+" "+str(dim[2])    

#warning may need to explicityly write the adiabatic parameter and correct gravitational parameters here
head3="1.66667E+00  0.00000E+00  1.00000E+00  0.00000E+00  0.00000E+00  0.00000E+00"

if ndim==2:
    head4="x y h m1 m2 e b1 b2 eb rhob bg1 bg2   gamma eta   grav1 grav2"
elif ndim==3:
    head4="x y z h m1 m2 m3 e b1 b2 b3 eb rhob bg1 bg2 bg3   gamma eta   grav1 grav2 grav3"

file.write(str.encode(header))
file.write(str.encode(head1+"\n"))     
file.write(str.encode(head2+"\n"))
file.write(str.encode(head3+"\n"))
file.write(str.encode(head4+"\n"))     
    
if ndim==3:    
    for i3 in range(dim[2]):
        for i2 in range(dim[1]):
            for i1 in range(dim[0]):
                line=""
                for j in range(nfields):
                    line=line+str(alldat[j,i3,i2,i1])
                line=line+"\n"
                file.write(str.encode(line))
                
if ndim==2:    
    for i2 in range(dim[1]):
        for i1 in range(dim[0]):
            line=""
            for j in range(nfields): 
                #print(j,i1,i2)
                line=line+" "+str(alldat[j,i2,i1])
            line=line+"\n"
            file.write(str.encode(line))          

In [5]:
file.close()